# Ettoday 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例：

1. 取出今天所有的新聞
2. 取出現在時間兩小時內的新聞
3. 根據範例，取出三天前下午三點到五點的新聞

## 1. 取出今天所有的新聞

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re

In [2]:
today = time.strftime("%Y/%m/%d",time.localtime())
print(today)

2020/01/01


In [22]:
url = 'https://www.ettoday.net/news/news-list.htm'

browser = webdriver.Chrome('./chromedriver')
browser.get(url)

In [4]:
# 每個兩秒鐘自動往下滑

import time
for i in range(10):
    time.sleep(2)
    browser.execute_script("window.scrollTo(0, 10000);")

In [5]:
html_source = browser.page_source
soup = BeautifulSoup(html_source, "html5lib")

for d in soup.find(class_="part_list_2").find_all('h3'):
    print(d.find(class_="date").text, d.find_all('a')[-1].text)

2020/01/01 17:18 日產前董座躲樂器箱逃黎巴嫩！　日方「看新聞才知」超錯愕
2020/01/01 17:17 獨／《康熙》製作人B2辦婚宴當節目做　廈門貨運華麗場景曝光
2020/01/01 17:17 黃明志不忍了！　驚曝《飆高音》：我寫過最後悔的歌...原因曝光
2020/01/01 17:16 雪地有裂縫！加拿大雪崩2死「16歲少年奇蹟獨活」逃命細節曝光
2020/01/01 17:16 林金結讚侯友宜：最強母雞　吳琪銘「護國保台演唱會」邀大支助陣
2020/01/01 17:14 父母離婚遭到處宣傳！小5女童「狂刺10刀」殺友報仇　殘忍手段點燃眾怒
2020/01/01 17:10 醉妹沙灘「脫男友褲賞鳥」　「搖柱起手勢」驚呆家長…激吻達陣瞬間慘遭上銬！
2020/01/01 17:05 澳洲野火狂燒！　國防部罕出動「喬勒斯號」救援受困居民
2020/01/01 17:05 日本研究：高速公路塞車維持原行駛車道有效紓緩車流
2020/01/01 17:00 加了鮮甜小章魚！首爾必吃麵疙瘩　還有烤青花魚外酥內嫩
2020/01/01 16:58 跨年夜上網數據暴增6成　電信業者統計：YouTube占近4成
2020/01/01 16:57 鄭朝方、賴清德大跳運動操　熱血鐵粉掛點滴也要相挺
2020/01/01 16:56 叼樹枝抓癢...撓完胸部放回草地　海鸚鵡被發現使用工具先例
2020/01/01 16:54 巴西龜見迷彩指甲狂抖「魔幻手指」　背後「羞羞意圖」網笑翻
2020/01/01 16:48 2019年最後一天　吉林天空驚現「3個太陽」奇景持續20分鐘
2020/01/01 16:47 游錫堃聲援《反滲透法》　左批國民黨「藍紅一體」、右嗆馬英九「居心可議」
2020/01/01 16:44 三勢力搶攻跨年人潮！柯文哲陪李旻蔚掃街　余天喊話：已替三重爭取6億經費
2020/01/01 16:43 好好先生性格大變！他失眠口渴又脹氣...中醫2招降伏爆跳男
2020/01/01 16:41 曬太陽被笑「那邊有肥貓」　店長裝箱眼神死：你才全家都肥！	
2020/01/01 16:39 國民黨退席反對《反滲透法》　蔡易餘：誰跟著中共跳腳一一現形
2020/01/01 16:38 2汪零下6度結伴翹家蹲路邊發抖！　暖司機開門安撫幫找家
2020

多了分類跟連結的寫法
html = driver.page_source
soup = BeautifulSoup(html, 'html5lib')
for h3 in soup.find(class_='part_list_2').find_all('h3'):
    if h3.find(class_='date').text.split(' ')[0] == today:
        Date = h3.find(class_='date').text
        Title = h3.find(name='a').text
        Category = h3.find(name='em').text
        Link = h3.find(name='a')['href']
        print(Date, Category, Title, 'https://www.ettoday.net'+Link)

### 2. 取出現在時間兩小時內的新聞

In [6]:
#把字串->時間->時戳 
def str_to_timestamp(strTime):
    string = str(strTime)
    timeArray = time.strptime(string, "%Y/%m/%d %H:%M")
    timestamp = time.mktime(timeArray)
    return timestamp

In [7]:
# 爬到的是字串－變成時戳－必須要大於 > 兩小時前的時戳 (小於就是比2小時前更早了)

two_hours_ago = float(time.time()) - 7200  # 兩小時前 = int(現在時戳秒數 - 7200秒)
Now = time.strftime('%Y/%m/%d %H:%M:%S',time.localtime())
Ago = time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(two_hours_ago))
print('現在時間:{}，獲取兩小時前:{}之後的新聞\n'.format(Now, Ago))

for h3 in soup.find(class_='part_list_2').find_all('h3'):
    date_string = h3.find(class_='date').text
    if str_to_timestamp(date_string) > two_hours_ago:
        Date = h3.find(class_='date').text
        Title = h3.find(name='a').text
        Category = h3.find(name='em').text
        Link = h3.find(name='a')['href']
        print(Date, Category, Title, 'https://www.ettoday.net'+Link)

現在時間:2020/01/01 17:19:03，獲取兩小時前:2020/01/01 15:19:03之後的新聞

2020/01/01 17:18 國際 日產前董座躲樂器箱逃黎巴嫩！　日方「看新聞才知」超錯愕 https://www.ettoday.net/news/20200101/1615075.htm
2020/01/01 17:17 影劇 獨／《康熙》製作人B2辦婚宴當節目做　廈門貨運華麗場景曝光 https://www.ettoday.net/news/20200101/1615284.htm
2020/01/01 17:17 影劇 黃明志不忍了！　驚曝《飆高音》：我寫過最後悔的歌...原因曝光 https://www.ettoday.net/news/20200101/1615274.htm
2020/01/01 17:16 國際 雪地有裂縫！加拿大雪崩2死「16歲少年奇蹟獨活」逃命細節曝光 https://www.ettoday.net/news/20200101/1615215.htm
2020/01/01 17:16 政治 林金結讚侯友宜：最強母雞　吳琪銘「護國保台演唱會」邀大支助陣 https://www.ettoday.net/news/20200101/1615017.htm
2020/01/01 17:14 國際 父母離婚遭到處宣傳！小5女童「狂刺10刀」殺友報仇　殘忍手段點燃眾怒 https://www.ettoday.net/news/20200101/1615133.htm
2020/01/01 17:10 國際 醉妹沙灘「脫男友褲賞鳥」　「搖柱起手勢」驚呆家長…激吻達陣瞬間慘遭上銬！ https://www.ettoday.net/news/20200101/1615232.htm
2020/01/01 17:05 軍武 澳洲野火狂燒！　國防部罕出動「喬勒斯號」救援受困居民 https://www.ettoday.net/news/20200101/1615154.htm
2020/01/01 17:05 ET車雲 日本研究：高速公路塞車維持原行駛車道有效紓緩車流 https://www.ettoday.net/news/20200101/1614400.htm
2020/01/01 17:00 旅遊 加了鮮甜小章魚！首爾必吃麵疙

## 3. 根據範例，取出三天前下午三點到五點的新聞

In [23]:
import datetime

#得到三天前的日期
today = datetime.date.today()
three_day_ago = today - datetime.timedelta(3)
print(three_day_ago)

2019-12-29


In [24]:
# 三天前15:00的時間戳
dt = "2019/12/29 15:00"
timeArray = time.strptime(dt, "%Y/%m/%d %H:%M")
three_clock = time.mktime(timeArray)

# 三天前17:00的時間戳
dt = "2019/12/29 17:00"
timeArray = time.strptime(dt, "%Y/%m/%d %H:%M")
five_clock = time.mktime(timeArray)


In [25]:
# 選到三天前的日期，按下查詢按鈕
from selenium.webdriver.support.ui import Select

selectYear = Select(browser.find_element_by_id("selY"))
selectYear.select_by_value('2019')
selectMonth = Select(browser.find_element_by_id("selM"))
selectMonth.select_by_value('12')
selectDay = Select(browser.find_element_by_id('selD'))
selectDay.select_by_value('29')

browser.find_element_by_css_selector('#button').click()

In [26]:
# 每2秒執行一次滾動視窗的JS程式碼
for i in range(10):
    time.sleep(2)
    browser.execute_script('window.scrollTo(1,10000);')

In [27]:
# 滾完了後，獲取html原始碼

html = browser.page_source
soup = BeautifulSoup(html, 'html5lib')
for h3 in soup.find(class_='part_list_2').find_all('h3'):
    date_string = h3.find(class_='date').text
    # 如果時戳比15:00大 and 比17:00小
    if five_clock > str_to_timestamp(date_string) > three_clock :
        Date = h3.find(class_='date').text
        Title = h3.find(name='a').text
        Category = h3.find(name='em').text
        Link = h3.find(name='a')['href']
        print(Date, Category, Title, 'https://www.ettoday.net'+Link)

In [28]:
browser.close()

### 第二種寫法

https://github.com/z655062/1st-PyCrawlerMarathon/blob/master/homework/Day021_HW.ipynb